In [11]:
"""MEV Specification: NL.

Swiss Metro Case Study
Demand Modeling: 1.202
Spring 2019
Alexandra Berke

"""

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models

pandas = pd.read_table("../Data/swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

# Specify parameters to estimate
#1: the name of the parameter,
#2: the default value,
#3: a lower bound (or None, if no bound is specified),
#4: an upper bound, (or None, if no bound is specified),
#5: a flag that indicates if the parameter must be estimated (0) or if it keeps its default value.
ASC_CAR	 = Beta('ASC_CAR',0,-20,20,0)
ASC_SBB	 = Beta('ASC_SBB',0,-10,10,1)
ASC_SM	 = Beta('ASC_SM',0,-20,20,0)
B_CAR_COST	 = Beta('B_CAR_COST',0,-10,10,0)
B_HE	 = Beta('B_HE',0,-10,10,0)
B_SM_COST	 = Beta('B_SM_COST',0,-10,10,0)
B_TRAIN_COST	 = Beta('B_TRAIN_COST',0,-10,10,0)
B_SENIOR	 = Beta('B_SENIOR',0,-10,10,0)
B_GA	 = Beta('B_GA',0,-10,80,0)
B_FIRST	 = Beta('B_FIRST',0,-10,80,0)
B_CAR_TIME1	 = Beta('B_CAR_TIME1',0,-1,1,0)
B_CAR_TIME2	 = Beta('B_CAR_TIME2',0,-1,1,0)
B_TRAIN_TIME1	 = Beta('B_TRAIN_TIME1',0,-1,1,0)
B_TRAIN_TIME2	 = Beta('B_TRAIN_TIME2',0,-1,1,0)
B_SM_TIME1	 = Beta('B_SM_TIME1',0,-1,1,0)
B_SM_TIME2	 = Beta('B_SM_TIME2',0,-1,1,0)
# parameters relevant to the nests
MU_rail = Beta('MU_rail',1,1,10,0)


# Define here arithmetic expressions for name that are not directly 
# available from the data

SENIOR  = DefineVariable('SENIOR', AGE   ==  5,database )
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)

# CAR_TT1  = DefineVariable('CAR_TT1',min(CAR_TT ,90), database)
# CAR_TT2  = DefineVariable('CAR_TT2',max(0, CAR_TT - 90 ), database)
CAR_TT1  = DefineVariable('CAR_TT1',((CAR_TT-0)*(CAR_TT<90)  + 90 * (CAR_TT >=90)), database)
CAR_TT2 = DefineVariable('CAR_TT2',((0)*(CAR_TT < 90) + (CAR_TT-90)*(CAR_TT >= 90)), database)
TRAIN_TT1  = DefineVariable('TRAIN_TT1',((TRAIN_TT-0)*(TRAIN_TT<90)  + 90 * (TRAIN_TT >=90)), database)
TRAIN_TT2 = DefineVariable('TRAIN_TT2',((0)*(TRAIN_TT < 90) + (TRAIN_TT-90)*(TRAIN_TT >= 90)), database)
SM_TT1  = DefineVariable('SM_TT1',((SM_TT-0)*(SM_TT<90)  + 90 * (SM_TT >=90)), database)
SM_TT2 = DefineVariable('SM_TT2',((0)*(SM_TT < 90) + (SM_TT-90)*(SM_TT >= 90)), database)

one  = DefineVariable('one',1,database)


#Utilities
Car_SP = ASC_CAR * one + B_CAR_TIME1 * CAR_TT1 + B_CAR_TIME2 * CAR_TT2 + B_CAR_COST * CAR_CO + B_SENIOR * SENIOR
SBB_SP = ASC_SBB * one + B_TRAIN_TIME1 * TRAIN_TT1 + B_TRAIN_TIME2 * TRAIN_TT2 + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE \
        + B_GA * GA + B_FIRST * FIRST
SM_SP = ASC_SM * one + B_SM_TIME1 * SM_TT1 + B_SM_TIME2 * SM_TT2 + B_SM_COST * SM_COST + B_HE * SM_HE + B_GA * GA \
        + B_SENIOR * SENIOR + B_FIRST * FIRST

V = {3: Car_SP,1: SBB_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Here we use the "biogeme" way for backward compatibility
exclude = (((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  )) + (  CHOICE   ==  0  ) + (  AGE == 6  ))>0
database.remove(exclude)

# Definition of nests
car = 1.0, [3]
rail = MU_rail, [1, 2]

nests = car, rail

# Estimation of the model
logprob = models.lognested(V,av,nests,CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "MEV_SM_NL"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

RuntimeError: src/biogeme.cc:301: Biogeme exception: Error for data entry 5048 : src/bioExprLiteral.cc:130: Biogeme exception: Value 8 out of range [0,18446744073709551615]